In [8]:
import pandas as pd
import numpy as np
import pathlib
import os
from glob import glob
from object_detection.utils import utils_frcnn

In [21]:
train_input = sorted(glob("/p/project/hai_hmgu/workspace/harsha/OrgaQuantData/train"+"/*",recursive=True))
train_annot_df = pd.read_csv("/p/project/hai_hmgu/workspace/harsha/OrgaQuantData/train_labels.csv", delimiter=';')
test_input = sorted(glob("/p/project/hai_hmgu/workspace/harsha/OrgaQuantData/test"+"/*",recursive=True))
test_annot_df = pd.read_csv("/p/project/hai_hmgu/workspace/harsha/OrgaQuantData/test_labels.csv", delimiter=',')

In [22]:
train_annot_df['newpath'] = [p.split('/')[-1] for p in train_annot_df.Path]
test_annot_df['newpath'] = [p.split('/')[-1] for p in test_annot_df.Path]

In [23]:
train_annot_df.shape,test_annot_df.shape

((13004, 7), (1135, 7))

In [24]:
train_annot_df = utils_frcnn.correct_annotdf(train_annot_df)
test_annot_df = utils_frcnn.correct_annotdf(test_annot_df)

train_annot_df.shape,test_annot_df.shape

((11468, 7), (1037, 7))

In [25]:
# Check if all the input images have corresponding annotations
new_train_input = []
for i in range(len(train_input)):
    path = train_input[i].split('/')[-1]
    if len(train_annot_df[train_annot_df['newpath'] == path]) != 0:
        new_train_input.append(train_input[i])

new_test_input = []
for i in range(len(test_input)):
    path = test_input[i].split('/')[-1]
    if len(test_annot_df[test_annot_df['newpath'] == path]) != 0:
        new_test_input.append(test_input[i])

In [26]:
import torch
from torch.utils.data.dataset import Dataset  # For custom data-sets
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
import random
import cv2
from glob import glob


class CustomDataset(Dataset):
    def __init__(self, img_path_list, annot_df, trans=None):
        self.norm = transforms.ToTensor()
        self.trans = trans
        self.img_path_list = img_path_list
        self.annot_df = annot_df

    def __getitem__(self, idx):
        # capture the image name and the full image path
        image_name = self.img_path_list[idx]
        # read the image
        image = cv2.imread(image_name)
        img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.norm(img_rgb) 

        # read the annotations
        path = image_name.split('/')[-1]
        tempdf = self.annot_df[self.annot_df.newpath == path]
        boxes = [tempdf.iloc[i,1:5].tolist() for i in range(len(tempdf))]
        if len(boxes) == 0:
            print(image_name)
        gtboxes = []
        for i in range(len(boxes)):
            if boxes[i][2] > boxes[i][0] and boxes[i][3] > boxes[i][1]:
                gtboxes.append(boxes[i])
            elif boxes[i][0] > boxes[i][2] and boxes[i][1] > boxes[i][3]:
                gtboxes.append([boxes[i][2],boxes[i][3],boxes[i][0],boxes[i][1]])
            elif boxes[i][0] > boxes[i][2] and boxes[i][3] > boxes[i][1]:
                gtboxes.append([boxes[i][2],boxes[i][1],boxes[i][0],boxes[i][3]])
            elif boxes[i][1] > boxes[i][3] and boxes[i][2] > boxes[i][0]:
                gtboxes.append([boxes[i][0],boxes[i][3],boxes[i][2],boxes[i][1]])
        labels = [1] * len(gtboxes)

        # bounding box to tensor
        boxes = torch.as_tensor(gtboxes, dtype=torch.float32)
        # labels to tensor
        labels = torch.as_tensor(labels, dtype=torch.int64)
        # prepare the final `target` dictionary
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        image = torch.as_tensor(image, dtype=torch.float32)
        

        return image, target


    def __len__(self):
        return len(self.img_path_list)

In [27]:
data_train = CustomDataset(new_train_input,train_annot_df)
data_test = CustomDataset(new_test_input,test_annot_df)

In [20]:
train_annot_df[train_annot_df.Path == "Subset_2_300x300_161.jpg"]

,Path,x1,y1,x2,y2,type


In [28]:
for i in range(len(data_train)):
    img,tgt = data_train[i]
    boxes = tgt["boxes"]
    if len(boxes) == 0:
        print(1)
                